# monitor_workflow_and_report_results

Top-level Notebook for monitoring and reporting results of workflow DRS data access scale tests.

In [128]:
import os
import time
from datetime import datetime
from pathlib import Path

import psutil

# Custom Setup for Execution Environment

## Setup for Michael's Broad laptop.

In [129]:
# Enable external development setup
external_development=True

In [130]:
# TODO Set this differently for running in Terra Jupyter Cloud Environment.
if external_development:
    NOTEBOOK_EXECUTION_DIRECTORY="/Users/mbaumann/Repos/mbaumann-broad/"
    %cd {NOTEBOOK_EXECUTION_DIRECTORY}

/Users/mbaumann/Repos/mbaumann-broad


In [131]:
if external_development:
    # The processing performed by this Notebook requires grep with PCRE
    # support, which the version of grep that comes with MacOS Monterey
    # does not provide. Therefore, install it with homebrew and add to PATH:
    # $ brew install grep
    os.environ['PATH'] = f"/usr/local/opt/grep/libexec/gnubin:{os.environ['PATH']}"

    # The processing performed by this Notebook requires sed "-z" option
    # support, which the version of sed that comes with MacOS Monterey
    # does not provide. Therefore, install it with homebrew and add to PATH:
    # $ brew install gnu-sed
    os.environ['PATH'] = f"/usr/local/opt/gnu-sed/libexec/gnubin:{os.environ['PATH']}"

    if os.environ.get('WORKSPACE_BUCKET') is None:
        # Workspace bucket used by: `DRS and Signed URL Development - Dev`
        WORKSPACE_BUCKET="gs://fc-b14e50ee-ccbe-4ee9-9aa4-f4e4ff85bc03"
        os.environ['WORKSPACE_BUCKET'] = WORKSPACE_BUCKET

# Manual Input/Configuration

Enter the workflow submission id, which is available from Job Manager
after the workflow is started.
For example:
```
 WF_SUBMISSION_ID="1a72b974-00c4-4316-86d5-7a7b1045f9ef"
```

In [132]:
#TODO Uncomment this line when using as a template Notebook
# WF_SUBMISSION_ID = "<Enter workflow submission id as shown above.>"
WF_SUBMISSION_ID="1a72b974-00c4-4316-86d5-7a7b1045f9ef"

Enter the start time for the workflow using UTC/GMT in this format:
```
2022/02/19 01:33:05
YYYY/MM/DD HH:MM:SS
```

For example, the current time in UTC in this format is:

In [133]:
datetime.utcnow().strftime("%Y/%m/%d %H:%M:%S")

'2022/03/20 21:23:35'

In [134]:
#TODO Uncomment this line when using as a template Notebook
# WF_START_TIME = "<Enter UTC date/time in the format above.>"
WF_START_TIME = datetime.utcnow().strftime("%Y/%m/%d %H:%M:%S")

## Processing Steps to Run

In [135]:
monitor_workflow=True
copy_workflow_logs=False
extract_timeseries_data=False
display_results=False

# Constants

In [136]:
WORKSPACE_BUCKET=os.environ['WORKSPACE_BUCKET']
WORKSPACE_BUCKET

'gs://fc-b14e50ee-ccbe-4ee9-9aa4-f4e4ff85bc03'

In [137]:
WS_WF_GS_URI=f"{WORKSPACE_BUCKET}/{WF_SUBMISSION_ID}"
WS_WF_GS_URI

'gs://fc-b14e50ee-ccbe-4ee9-9aa4-f4e4ff85bc03/1a72b974-00c4-4316-86d5-7a7b1045f9ef'

In [138]:
TEST_RESULTS_DIR=Path('./test_results').resolve().as_posix()
! mkdir -p {TEST_RESULTS_DIR}
TEST_RESULTS_DIR

'/Users/mbaumann/Repos/mbaumann-broad/test_results'

In [139]:
WF_TEST_RESULTS_DIR=os.path.join(TEST_RESULTS_DIR, f"submission_{WF_SUBMISSION_ID}")
! mkdir -p {WF_TEST_RESULTS_DIR}
WF_TEST_RESULTS_DIR

'/Users/mbaumann/Repos/mbaumann-broad/test_results/submission_1a72b974-00c4-4316-86d5-7a7b1045f9ef'

In [140]:
MONITORING_OUTPUT_DIR=os.path.join(WF_TEST_RESULTS_DIR,
                                   f"monitoring_data_{datetime.strptime(WF_START_TIME, '%Y/%m/%d %H:%M:%S').strftime('%Y%m%d_%H%M%S')}")
! mkdir -p {MONITORING_OUTPUT_DIR}
MONITORING_OUTPUT_DIR

'/Users/mbaumann/Repos/mbaumann-broad/test_results/submission_1a72b974-00c4-4316-86d5-7a7b1045f9ef/monitoring_data_20220320_212335'

In [141]:
SCRIPTS=Path("./terra-workflow-scale-test-tools/scripts").resolve().as_posix()
SCRIPTS

'/Users/mbaumann/Repos/mbaumann-broad/terra-workflow-scale-test-tools/scripts'

# Functions

In [142]:
def start_monitoring_background_process() -> psutil.Process:
    print("Starting monitoring background process ...")
    process = psutil.Popen(["python3", f"{SCRIPTS}/monitor_response_times.py", "--output-dir", f"{MONITORING_OUTPUT_DIR}" ])
    print(f"Started {process}")
    return process

In [143]:
def stop_monitoring_background_process(process: psutil.Process) -> None:
    print("Stopping monitoring background process ...")
    process.terminate()
    process.wait(60)
    print("Stopped monitoring background process.")

In [144]:
def wait_for_workflow_to_complete() -> None:
    # TODO Implement this
    pretend_duration = 60
    print(f"Pretending to wait for workflow to complete by sleeping {pretend_duration} seconds ...")
    time.sleep(pretend_duration)
    print("Pretending workflow is complete!")

# Monitor response times during workflow execution

In [145]:
if monitor_workflow:
    monitoring_process = start_monitoring_background_process()

    wait_for_workflow_to_complete()

    stop_monitoring_background_process(monitoring_process)

Starting monitoring background process ...
Started psutil.Popen(pid=90895, name='python3.9', status='running', started='14:23:36')
Pretending to wait for workflow to complete by sleeping 60 seconds ...
Pretending workflow is complete!
Stopping monitoring background process ...
Stopped monitoring background process.


# Copy workflow logs from the workspace bucket to the local filesystem

In [146]:
if copy_workflow_logs:
    workflow_logs_dir = f"{WF_TEST_RESULTS_DIR}/workflow-logs"
    if not Path(workflow_logs_dir).exists():
        ! {SCRIPTS}/copy_workflow_logs_to_local_fs.sh {WS_WF_GS_URI} {workflow_logs_dir}
    else:
        print(f"The workflow-logs directory already exists: {workflow_logs_dir}")
        print("Skipping copy of the workflow logs.")


In [147]:
if extract_timeseries_data:
    ! {SCRIPTS}/extract_drs_localization_timestamps_from_local_fs.sh {WF_TEST_RESULTS_DIR}
    

In [148]:
if extract_timeseries_data:
    ! {SCRIPTS}/extract_drs_localization_fallback_timestamps_from_local_fs.sh {WF_TEST_RESULTS_DIR}

# Display the results of the workflow run

## Workflow DRS localization rates

In [149]:
if display_results:
    %run {SCRIPTS}/graph_drs_data_access_rates.ipynb
    f"{WF_TEST_RESULTS_DIR}/drs_localization_timeseries.tsv"
    f"{WF_TEST_RESULTS_DIR}/drs_localization_fallback_timeseries.tsv"

## Service/endpoint response times

In [150]:
if display_results:
    %run {SCRIPTS}/graph_response_time_data.ipynb TODO